<a href="https://colab.research.google.com/github/mandumonster/-NLPTeam/blob/main/bpe_word_piece_bert_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**1. BPE**


# 문장 딕셔너리 형태로 변환

In [ ]:
import re
import collections

In [ ]:
from collections import defaultdict

def preprocess(text):
    # 불용어 등 처리하는 자리
    return text

def get_dictionary(corpus):
    dictionary = defaultdict(int)
    for line in corpus:
        tokens = preprocess(line).strip().split(" ")
        for token in tokens:
            dictionary[" ".join(list(token)) + " </w>"] += 1

    return dict(dictionary)

# 예시 문장 데이터
corpus = [
    "low low low low low lower newest lower newest newest newest newest newest widest widest widest"
]

# 단어 사전 생성
vocab = get_dictionary(corpus)

# 결과 출력
print(vocab)

{'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w e s t </w>': 6, 'w i d e s t </w>': 3}


# 10번을 반복하며 단어쌍 생성

​1. get_stats 함수:

   - `vocab`이라는 단어 빈도수 사전을 입력으로 받는다.

   - 각 단어를 공백으로 분리하여 부분 단어(subwords)의 빈도수를 계산한다.

   - 예를 들어, 'l o w </w>'가 입력으로 주어지면 'l', 'o', 'w', '</w>'의 빈도수를 계산하고, 'l o', 'o w', 'w </w>'의 빈도수를 계산한다.

   - 이 정보를 이용하여 부분 단어 쌍의 빈도수를 저장한 딕셔너리인 `pairs`를 반환한다.

In [ ]:
# 부분 단어 쌍의 빈도수를 계산하는 함수
def get_stats(vocab):
    pairs = collections.defaultdict(int)

    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i], symbols[i+1]] += freq

    return pairs

In [ ]:
# 가장 높은 빈도수를 가진 부분 단어 쌍으로 단어를 합치는 함수
def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')

    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]

    return v_out


2. merge_vocab 함수:
   - `pair`는 합쳐질 부분 단어 쌍을 나타낸다.

   - `v_in`은 단어 빈도수 사전이다.

   - 주어진 부분 단어 쌍을 가지고 단어 빈도수 사전을 업데이트한다.

   - 새로운 단어를 생성하고, 이를 이용하여 기존 단어를 업데이트한다.

   - 업데이트된 단어 빈도수 사전을 반환한다.

In [ ]:
# 반복 횟수
num_merges = 10

# 주어진 횟수만큼 반복
for i in range(num_merges):
    # 부분 단어 쌍의 빈도수 계산
    pairs = get_stats(vocab)

    # 가장 높은 빈도수를 가진 부분 단어 쌍 선택
    best = max(pairs, key=pairs.get)

    # 선택된 부분 단어 쌍으로 단어 빈도수 사전 업데이트
    vocab = merge_vocab(best, vocab)

    # 선택된 부분 단어 쌍 출력
    print(best)

('e', 's')
('es', 't')
('est', '</w>')
('l', 'o')
('lo', 'w')
('n', 'e')
('ne', 'w')
('new', 'est</w>')
('low', '</w>')
('w', 'i')


#**2. BertTokenizer 임포트**
* "bert-base-uncased"라는 사전 학습된 BERT 모델의 가중치와 어휘 가져오기

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

print(tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


# BertTokenizer의 어휘리스트 확인하기

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# 어휘에서 2글자 이상의 단어만 가져옴
vocab_list = [token for token in tokenizer.get_vocab().keys() if token.isalpha() and len(token) >= 2]

# 어휘 리스트 앞부터 100개 가져오기
print(vocab_list[:100])

['the', 'of', 'and', 'in', 'to', 'was', 'he', 'is', 'as', 'for', 'on', 'with', 'that', 'it', 'his', 'by', 'at', 'from', 'her', 'she', 'you', 'had', 'an', 'were', 'but', 'be', 'this', 'are', 'not', 'my', 'they', 'one', 'which', 'or', 'have', 'him', 'me', 'first', 'all', 'also', 'their', 'has', 'up', 'who', 'out', 'been', 'when', 'after', 'there', 'into', 'new', 'two', 'its', 'time', 'would', 'no', 'what', 'about', 'said', 'we', 'over', 'then', 'other', 'so', 'more', 'can', 'if', 'like', 'back', 'them', 'only', 'some', 'could', 'where', 'just', 'during', 'before', 'do', 'made', 'school', 'through', 'than', 'now', 'years', 'most', 'world', 'may', 'between', 'down', 'well', 'three', 'year', 'while', 'will', 'later', 'city', 'under', 'around', 'did', 'such']


* 어휘 리스트 중 있는 단어를 사용하여 Tokenizer가 정상작동 하는지 확인

In [ ]:
tokenizer.tokenize("the forest")

['the', 'forest']

In [ ]:
tokenizer.tokenize("theforest")

['the', '##for', '##est']